# Importer le client

In [1]:
from google.cloud import bigquery

# Spécifiez le chemin vers votre fichier JSON
client = bigquery.Client.from_service_account_json('primal-pod-401712-dcbbeb5f006a.json')

# Créer un ensemble de données

In [2]:
from google.cloud import bigquery

dataset_id = "{}.datacampSongs".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

dataset.location = "EU"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset primal-pod-401712.datacampSongs


# Créer la table Songs

In [10]:
from google.cloud import bigquery

# Spécifiez le chemin vers votre fichier JSON
client = bigquery.Client.from_service_account_json('primal-pod-401712-dcbbeb5f006a.json') 

# ID du dataset et de la table
dataset_id = "{}.datacampSongs".format(client.project)
table_id = "{}.Songs".format(dataset_id)

# Définition du schéma de la table
schema = [
    bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("Name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Artist", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Album", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Cluster", "INT64", mode="NULLABLE")
]

# Création de la table dans BigQuery
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # API request
print("Created table {}.{}".format(table.project, table.table_id))


Created table primal-pod-401712.Songs


In [11]:
from google.cloud import bigquery


# ID du dataset et de la table
dataset_id = "{}.datacampSongs".format(client.project)
table_id = "{}.Songs".format(dataset_id)

table = client.get_table(table_id)  # Make an API request.

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

Got table 'primal-pod-401712.datacampSongs.Songs'.
Table schema: [SchemaField('id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('Name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Artist', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Album', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Cluster', 'INTEGER', 'NULLABLE', None, None, (), None)]
Table description: None
Table has 0 rows


In [13]:
import pandas as pd

df = pd.read_csv('dataset_cluster.csv')
df.head()

,ID,Name,Artist,Album,Cluster_Labels
0,0,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,17
1,1,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),17
2,2,Bridge Over Troubled Water,Simon & Garfunkel,Bridge Over Troubled Water,19
3,3,All Along the Watchtower,Jimi Hendrix,Electric Ladyland,4
4,4,In the Ghetto,Elvis Presley,Elvis 30 #1 Hits,9


In [16]:
from google.cloud import bigquery

# Spécifiez le chemin vers votre fichier JSON
client = bigquery.Client.from_service_account_json('primal-pod-401712-dcbbeb5f006a.json') 

# ID du dataset et de la table
dataset_id = "{}.datacampSongs".format(client.project)
table_id = "{}.Songs".format(dataset_id)

# Spécifiez le chemin vers votre fichier CSV
file_path = "dataset_cluster.csv"

# Configuration de l'importation
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Pour sauter l'en-tête du fichier CSV
    autodetect=True,  # Pour détecter automatiquement le type de données et le schéma
    schema=[
        bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
        bigquery.SchemaField("Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Artist", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Album", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Cluster", "INT64", mode="NULLABLE")  # Mapping Cluster_Labels du CSV à Cluster dans la DB
    ]
)

# Charger le fichier CSV dans BigQuery
with open(file_path, "rb") as csv_file:
    load_job = client.load_table_from_file(
        csv_file, table_id, job_config=job_config
    )
    load_job.result()  # Attendez que le job d'importation soit terminé

print(f"Loaded {load_job.output_rows} rows into {table_id}.")


Loaded 471 rows into primal-pod-401712.datacampSongs.Songs.
